# The Cancer Gene Atlas Cancer (TCGA) - Esophageal Carcinoma (ESCA)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
seaborn 0.7.1
Wed Apr 25 17:36:54 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Esophageal carcinoma [ESCA]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-IG-A51D,1919,ESCA,TCGA,F8DBAB24-B9F4-4B8A-BFEA-57856CCF6364,IG,63.0,NaN,272,Nationwide Children's Hospital,M1,...,19.9,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-LN-A9FR,1920,ESCA,TCGA,CFDC8F4D-A25B-4C63-9832-58444581A505,LN,70.0,NaN,395,Nationwide Children's Hospital,NaN,...,20.6,NaN,Never,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-LN-A7HZ,1921,ESCA,TCGA,EF8BEC6F-F594-4076-9F38-157165633EF1,LN,49.0,NaN,362,Nationwide Children's Hospital,NaN,...,20.0,24.0,Never,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-LN-A7HY,1922,ESCA,TCGA,7A32847F-69BB-4592-8FEB-CFD5D355ADA9,LN,50.0,NaN,362,Nationwide Children's Hospital,NaN,...,21.0,NaN,Never,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-Z6-A8JE,1923,ESCA,TCGA,1116DDAE-76B0-48D6-869C-BE65D8B09D35,Z6,57.0,NaN,382,Nationwide Children's Hospital,M0,...,21.6,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(185, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_ESCA_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,973a12ac-4778-4797-bafd-1b688c175920,17dc572d-0ecb-4e15-9629-f75426451f86,adad2b3f-4d90-4324-b0e1-e57fe7f8e820,9941f93e-8cba-4f88-b721-f49c2bd6d5dc,696b61a8-5384-4e21-bdc2-a17767f17753,42516944-c5d6-4520-8004-ccf591155d93,f49f6249-41e6-424c-800d-9ad5135417cd,b47d4eea-4457-4619-a12a-1e3b48efacfa,af5f6f2d-b495-41bb-bb39-dc82b1a19e3e,b12ca178-5954-4d6e-b136-db4fefd5edfd,...,35a96219-b612-4d79-b4d5-4d020e8da6d6,13d1c14c-496b-4d6f-a86a-fdffd5188d2c,80f9a9e1-e747-4d94-85af-1062a3c13668,82e2a63f-d9d0-474e-a3f3-038375f931ed,9148908a-2bfe-438f-b6de-8e41aea3c8ae,1c17d97f-822d-42fe-986b-c9fde75e0f9b,8aed2c63-35a3-4987-9d84-5651c5e12477,c6fc4b79-4135-49d8-9b52-7941b74f363b,5c523d4c-4b44-48b6-87cb-3ea349c8add6,00be0efb-29a5-418e-a5c7-df8af0d127bb
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,128.691000,80.38160,65.162500,98.831900,34.109800,1.893670e+02,137.790000,51.02390,60.861700,108.91900,...,56.284800,134.96200,51.519700,68.791400,82.973200,65.073800,61.642800,106.490000,77.147300,107.08900
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,32.123400,26.48310,24.030400,44.795500,7.422990,1.248090e-09,55.898800,68.40160,31.493200,37.05770,...,0.825797,11.99070,19.945700,1.000160,13.975400,42.488400,0.904002,31.646900,40.677500,19.67610
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,7.585860,7.71806,30.338800,14.018600,8.526330,8.989110e+00,10.769700,19.08330,12.771800,9.35125,...,8.222250,13.30620,16.642900,20.513300,20.557100,7.696030,16.959900,17.626700,31.281100,19.67060
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,71.397400,36.75370,41.126600,95.151600,49.058200,4.429750e+01,32.185200,58.17440,36.926900,50.83910,...,24.037800,30.44830,20.324500,18.809700,26.386400,68.421700,49.804300,88.656100,59.131200,36.65790
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.049726,1.14849,0.153385,0.176444,0.847696,1.335130e-01,0.978689,2.76539,0.792514,1.59534,...,2.067960,1.47842,0.690244,0.966935,0.097739,0.580355,0.000000,0.406389,0.068688,2.65627


In [19]:
matrix.shape

(199169, 198)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,973a12ac-4778-4797-bafd-1b688c175920,17dc572d-0ecb-4e15-9629-f75426451f86,adad2b3f-4d90-4324-b0e1-e57fe7f8e820,9941f93e-8cba-4f88-b721-f49c2bd6d5dc,696b61a8-5384-4e21-bdc2-a17767f17753,42516944-c5d6-4520-8004-ccf591155d93,f49f6249-41e6-424c-800d-9ad5135417cd,b47d4eea-4457-4619-a12a-1e3b48efacfa,af5f6f2d-b495-41bb-bb39-dc82b1a19e3e,b12ca178-5954-4d6e-b136-db4fefd5edfd,...,35a96219-b612-4d79-b4d5-4d020e8da6d6,13d1c14c-496b-4d6f-a86a-fdffd5188d2c,80f9a9e1-e747-4d94-85af-1062a3c13668,82e2a63f-d9d0-474e-a3f3-038375f931ed,9148908a-2bfe-438f-b6de-8e41aea3c8ae,1c17d97f-822d-42fe-986b-c9fde75e0f9b,8aed2c63-35a3-4987-9d84-5651c5e12477,c6fc4b79-4135-49d8-9b52-7941b74f363b,5c523d4c-4b44-48b6-87cb-3ea349c8add6,00be0efb-29a5-418e-a5c7-df8af0d127bb
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,128.691000,80.38160,65.162500,98.831900,34.109800,1.893670e+02,137.790000,51.02390,60.861700,108.91900,...,56.284800,134.96200,51.519700,68.791400,82.973200,65.073800,61.642800,106.490000,77.147300,107.08900
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,32.123400,26.48310,24.030400,44.795500,7.422990,1.248090e-09,55.898800,68.40160,31.493200,37.05770,...,0.825797,11.99070,19.945700,1.000160,13.975400,42.488400,0.904002,31.646900,40.677500,19.67610
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,7.585860,7.71806,30.338800,14.018600,8.526330,8.989110e+00,10.769700,19.08330,12.771800,9.35125,...,8.222250,13.30620,16.642900,20.513300,20.557100,7.696030,16.959900,17.626700,31.281100,19.67060
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,71.397400,36.75370,41.126600,95.151600,49.058200,4.429750e+01,32.185200,58.17440,36.926900,50.83910,...,24.037800,30.44830,20.324500,18.809700,26.386400,68.421700,49.804300,88.656100,59.131200,36.65790
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.049726,1.14849,0.153385,0.176444,0.847696,1.335130e-01,0.978689,2.76539,0.792514,1.59534,...,2.067960,1.47842,0.690244,0.966935,0.097739,0.580355,0.000000,0.406389,0.068688,2.65627


In [22]:
matrix.shape

(79930, 198)

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

,TCGA-IG-A3QL-01,TCGA-IG-A3YA-01,TCGA-L5-A43I-01,TCGA-LN-A49V-01,TCGA-LN-A49R-01,TCGA-LN-A49P-01,TCGA-Q9-A6FU-01,TCGA-LN-A5U7-01,TCGA-LN-A49X-01,TCGA-LN-A49U-01,...,TCGA-L5-A8NW-01,TCGA-JY-A93F-01,TCGA-JY-A93E-01,TCGA-JY-A938-01,TCGA-2H-A9GR-01,TCGA-2H-A9GQ-01,TCGA-2H-A9GO-01,TCGA-V5-A7RC-06,TCGA-IG-A3YB-01,TCGA-L5-A88T-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,128.691000,80.38160,65.162500,98.831900,34.109800,1.893670e+02,137.790000,51.02390,60.861700,108.91900,...,56.284800,134.96200,51.519700,68.791400,82.973200,65.073800,61.642800,106.490000,77.147300,107.08900
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,32.123400,26.48310,24.030400,44.795500,7.422990,1.248090e-09,55.898800,68.40160,31.493200,37.05770,...,0.825797,11.99070,19.945700,1.000160,13.975400,42.488400,0.904002,31.646900,40.677500,19.67610
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,7.585860,7.71806,30.338800,14.018600,8.526330,8.989110e+00,10.769700,19.08330,12.771800,9.35125,...,8.222250,13.30620,16.642900,20.513300,20.557100,7.696030,16.959900,17.626700,31.281100,19.67060
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,71.397400,36.75370,41.126600,95.151600,49.058200,4.429750e+01,32.185200,58.17440,36.926900,50.83910,...,24.037800,30.44830,20.324500,18.809700,26.386400,68.421700,49.804300,88.656100,59.131200,36.65790
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.049726,1.14849,0.153385,0.176444,0.847696,1.335130e-01,0.978689,2.76539,0.792514,1.59534,...,2.067960,1.47842,0.690244,0.966935,0.097739,0.580355,0.000000,0.406389,0.068688,2.65627


# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-IG-A3QL-01,TCGA-IG-A3YA-01,TCGA-L5-A43I-01,TCGA-LN-A49V-01,TCGA-LN-A49R-01,TCGA-LN-A49P-01,TCGA-Q9-A6FU-01,TCGA-LN-A5U7-01,TCGA-LN-A49X-01,TCGA-LN-A49U-01,...,TCGA-L5-A8NW-01,TCGA-JY-A93F-01,TCGA-JY-A93E-01,TCGA-JY-A938-01,TCGA-2H-A9GR-01,TCGA-2H-A9GQ-01,TCGA-2H-A9GO-01,TCGA-V5-A7RC-06,TCGA-IG-A3YB-01,TCGA-L5-A88T-01
ARF5,128.691000,80.38160,65.162500,98.831900,34.109800,1.893670e+02,137.790000,51.02390,60.861700,108.91900,...,56.284800,134.96200,51.519700,68.791400,82.973200,65.073800,61.642800,106.490000,77.147300,107.08900
M6PR,32.123400,26.48310,24.030400,44.795500,7.422990,1.248090e-09,55.898800,68.40160,31.493200,37.05770,...,0.825797,11.99070,19.945700,1.000160,13.975400,42.488400,0.904002,31.646900,40.677500,19.67610
ESRRA,7.585860,7.71806,30.338800,14.018600,8.526330,8.989110e+00,10.769700,19.08330,12.771800,9.35125,...,8.222250,13.30620,16.642900,20.513300,20.557100,7.696030,16.959900,17.626700,31.281100,19.67060
FKBP4,71.397400,36.75370,41.126600,95.151600,49.058200,4.429750e+01,32.185200,58.17440,36.926900,50.83910,...,24.037800,30.44830,20.324500,18.809700,26.386400,68.421700,49.804300,88.656100,59.131200,36.65790
CYP26B1,0.049726,1.14849,0.153385,0.176444,0.847696,1.335130e-01,0.978689,2.76539,0.792514,1.59534,...,2.067960,1.47842,0.690244,0.966935,0.097739,0.580355,0.000000,0.406389,0.068688,2.65627


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_esca_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining mssing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-IG-A3QL-01,TCGA-IG-A3YA-01,TCGA-L5-A43I-01,TCGA-LN-A49V-01,TCGA-LN-A49R-01,TCGA-LN-A49P-01,TCGA-Q9-A6FU-01,TCGA-LN-A5U7-01,TCGA-LN-A49X-01,TCGA-LN-A49U-01,...,TCGA-L5-A8NW-01,TCGA-JY-A93F-01,TCGA-JY-A93E-01,TCGA-JY-A938-01,TCGA-2H-A9GR-01,TCGA-2H-A9GQ-01,TCGA-2H-A9GO-01,TCGA-V5-A7RC-06,TCGA-IG-A3YB-01,TCGA-L5-A88T-01
A1BG,0.047658,0.047658,0.047658,0.093696,0.012631,0.047658,0.015026,0.047658,0.069135,0.176370,...,0.047658,0.047658,0.020573,0.047658,0.047658,0.047658,0.047658,0.036011,0.021093,0.047658
A1CF,0.021281,0.061991,3.580900,0.044032,0.040689,0.028597,0.029253,0.043422,0.025783,0.064788,...,0.135857,0.029585,0.139604,0.530435,1.790830,0.042395,1.729950,0.126760,0.057037,0.195839
A2M,40.756100,233.358000,63.566800,45.110700,63.001500,209.073000,65.437500,32.551500,219.891000,33.737900,...,167.477000,8.026310,316.762000,164.549000,214.832000,207.817000,21.669100,160.279000,162.917000,234.902000
A2ML1,11.454900,29.593800,0.144604,0.198167,7.513420,12.439700,6.547130,17.242900,2.219060,16.332400,...,357.996000,125.034000,0.178490,1.625260,0.176050,0.551188,0.123142,0.531574,13.781400,834.340000
A3GALT2,0.013181,0.161606,0.116826,0.014365,0.041506,0.067298,0.376633,0.066942,0.082658,0.202716,...,0.316258,0.109605,0.039680,0.109605,0.081780,0.112796,0.109605,0.017065,0.054635,0.085016


In [34]:
normalized_matrix.shape

(19412, 198)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-IG-A3QL-01,TCGA-IG-A3YA-01,TCGA-L5-A43I-01,TCGA-LN-A49V-01,TCGA-LN-A49R-01,TCGA-LN-A49P-01,TCGA-Q9-A6FU-01,TCGA-LN-A5U7-01,TCGA-LN-A49X-01,TCGA-LN-A49U-01,...,TCGA-L5-A8NW-01,TCGA-JY-A93F-01,TCGA-JY-A93E-01,TCGA-JY-A938-01,TCGA-2H-A9GR-01,TCGA-2H-A9GQ-01,TCGA-2H-A9GO-01,TCGA-V5-A7RC-06,TCGA-IG-A3YB-01,TCGA-L5-A88T-01
A1BG,-1.076291,-1.079145,-0.978350,-0.758087,-1.669663,-1.047646,-1.598248,-1.052143,-0.939902,-0.530963,...,-1.104528,-1.117917,-1.417643,-1.001916,-1.095951,-1.094156,-1.072629,-1.210661,-1.458717,-1.118166
A1CF,-1.426430,-0.964958,0.897502,-1.086043,-1.161605,-1.269468,-1.308910,-1.092577,-1.368268,-0.965895,...,-0.649586,-1.324979,-0.586034,0.044577,0.478963,-1.144984,0.487265,-0.664111,-1.026694,-0.504406
A2M,1.855762,2.610739,2.146740,1.924470,2.028268,2.594513,2.040744,1.782288,2.562606,1.750730,...,2.441287,1.108459,2.769801,2.536240,2.558008,2.545385,1.585072,2.437783,2.429115,2.574581
A2ML1,1.304561,1.713917,-0.496310,-0.432778,1.104755,1.369025,1.040968,1.506327,0.566568,1.435662,...,2.771210,2.300971,-0.479318,0.530868,-0.528455,-0.030996,-0.660362,-0.041530,1.356442,3.125038
A3GALT2,-1.634469,-0.548827,-0.588950,-1.572489,-1.152970,-0.897786,-0.199165,-0.904584,-0.862319,-0.470500,...,-0.282626,-0.756225,-1.132357,-0.640224,-0.861443,-0.720002,-0.710937,-1.534982,-1.045383,-0.866806


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-IG-A3QL-01,TCGA-IG-A3YA-01,TCGA-L5-A43I-01,TCGA-LN-A49V-01,TCGA-LN-A49R-01,TCGA-LN-A49P-01,TCGA-Q9-A6FU-01,TCGA-LN-A5U7-01,TCGA-LN-A49X-01,TCGA-LN-A49U-01,...,TCGA-L5-A8NW-01,TCGA-JY-A93F-01,TCGA-JY-A93E-01,TCGA-JY-A938-01,TCGA-2H-A9GR-01,TCGA-2H-A9GQ-01,TCGA-2H-A9GO-01,TCGA-V5-A7RC-06,TCGA-IG-A3YB-01,TCGA-L5-A88T-01
A1BG,-0.937497,-1.093755,-0.984324,-0.743012,-1.464457,-1.285958,-1.445574,-0.959883,-0.898131,-0.507572,...,-1.249767,-0.950402,-1.486041,-1.272481,-1.070736,-1.153610,-1.142714,-1.224663,-1.363996,-1.032431
A1CF,-1.255289,-0.985622,0.807142,-1.047131,-1.068815,-1.578695,-1.181250,-0.992571,-1.289898,-0.935899,...,-0.751686,-1.158122,-0.690408,-0.129081,0.521722,-1.205024,0.555398,-0.731035,-0.951718,-0.421964
A2M,1.836310,2.597037,2.039569,1.769722,2.121334,2.517729,1.902505,1.643104,2.511808,1.666194,...,2.469650,1.238841,2.791371,2.503177,2.674444,2.554827,1.633736,2.396923,2.593293,2.566254
A2ML1,1.308844,1.663148,-0.486996,-0.416467,1.242518,1.297358,0.919377,1.364065,0.487362,1.351977,...,2.762174,2.337112,-0.590100,0.364027,-0.533332,-0.145589,-0.685348,-0.158570,1.462079,3.065532
A3GALT2,-1.446188,-0.616119,-0.585881,-1.470772,-1.060814,-1.100019,-0.181527,-0.817087,-0.820681,-0.451682,...,-0.387232,-0.534288,-1.207809,-0.838343,-0.851237,-0.786527,-0.743249,-1.520308,-0.971107,-0.780975


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-IG-A3QL-01,TCGA-IG-A3YA-01,TCGA-L5-A43I-01,TCGA-LN-A49V-01,TCGA-LN-A49R-01,TCGA-LN-A49P-01,TCGA-Q9-A6FU-01,TCGA-LN-A5U7-01,TCGA-LN-A49X-01,TCGA-LN-A49U-01,...,TCGA-L5-A8NW-01,TCGA-JY-A93F-01,TCGA-JY-A93E-01,TCGA-JY-A938-01,TCGA-2H-A9GR-01,TCGA-2H-A9GQ-01,TCGA-2H-A9GO-01,TCGA-V5-A7RC-06,TCGA-IG-A3YB-01,TCGA-L5-A88T-01
A1BG,0.888159,0.000856,0.622257,1.992541,-2.104171,-1.090564,-1.996943,0.761043,1.111701,3.329483,...,-0.885056,0.814881,-2.226736,-1.014038,0.131565,-0.339030,-0.277157,-0.742502,-1.533703,0.349080
A1CF,-0.598616,-0.224854,2.259934,-0.310106,-0.340160,-1.046859,-0.495997,-0.234485,-0.646584,-0.155938,...,0.099384,-0.463941,0.184315,0.962320,1.864340,-0.528948,1.911015,0.128006,-0.177863,0.556381
A2M,-0.702912,0.992813,-0.249831,-0.851343,-0.067571,0.816027,-0.555358,-1.133586,0.802830,-1.082115,...,0.708855,-2.034723,1.425998,0.783590,1.165359,0.898722,-1.154467,0.546741,0.984465,0.924193
A2ML1,0.355262,0.563944,-0.702474,-0.660933,0.316197,0.348497,0.125869,0.387786,-0.128585,0.380667,...,1.211262,0.960904,-0.763202,-0.201228,-0.729766,-0.501388,-0.819302,-0.509034,0.445516,1.389938
A3GALT2,-1.913221,0.593766,0.685089,-1.987469,-0.749308,-0.867716,1.906327,-0.013203,-0.024057,1.090401,...,1.285054,0.840911,-1.193264,-0.077400,-0.116341,0.079097,0.209806,-2.137079,-0.478374,0.095866


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(19412, 198)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

(18967, 198)

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

(18875, 198)

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [52]:
filename = path+'tcga_esca_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18875, 2)

# Save Gene List

In [56]:
filename = path+'tcga_esca_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-2H-A9GF-01,67,MALE,M0,N1,T3,Stage III,"Esophagus Adenocarcinoma, NOS",Primary Solid Tumor,Esophagus,G3
TCGA-2H-A9GG-01,66,MALE,M0,N1,T3,Stage III,"Esophagus Adenocarcinoma, NOS",Primary Solid Tumor,Esophagus,G2
TCGA-2H-A9GH-01,44,MALE,M0,N1,T1,Stage IIB,"Esophagus Adenocarcinoma, NOS",Primary Solid Tumor,Esophagus,G2
TCGA-2H-A9GI-01,68,MALE,M0,N1,T3,Stage III,"Esophagus Adenocarcinoma, NOS",Primary Solid Tumor,Esophagus,G2
TCGA-2H-A9GJ-01,57,MALE,M0,N0,T1,Stage I,"Esophagus Adenocarcinoma, NOS",Primary Solid Tumor,Esophagus,G2


In [60]:
attribute_list.shape

(198, 10)

# Save Attribute List

In [61]:
filename = path+'tcga_esca_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [62]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [63]:
standard_matrix.head()

,TCGA-2H-A9GF-01,TCGA-2H-A9GG-01,TCGA-2H-A9GH-01,TCGA-2H-A9GI-01,TCGA-2H-A9GJ-01,TCGA-2H-A9GK-01,TCGA-2H-A9GL-01,TCGA-2H-A9GM-01,TCGA-2H-A9GN-01,TCGA-2H-A9GO-01,...,TCGA-VR-AA7D-01,TCGA-VR-AA7I-01,TCGA-X8-AAAR-01,TCGA-XP-A8T6-01,TCGA-XP-A8T8-01,TCGA-Z6-A8JD-01,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-ZR-A9CJ-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0.090719,-0.717083,0.060388,-0.303100,-0.343498,0.403810,-0.666609,0.040210,0.070541,-0.161770,...,-0.434386,0.191715,-0.060722,0.020033,-0.838258,-0.919038,-0.010243,0.494729,-0.363695,-0.494964
A1CF,-0.310919,0.962118,0.618336,0.537458,0.941894,0.062686,0.992437,0.375763,0.739663,0.678998,...,-0.846086,-0.634077,0.193974,-0.411909,-0.048367,-0.563413,-0.331131,0.183881,-0.795605,0.496985
A2M,0.343729,0.697490,0.252810,-0.100614,-0.272295,0.818828,0.798592,0.454894,0.727834,-0.827629,...,-0.686238,0.242720,0.758141,0.394257,-0.696358,0.030666,-0.534845,-0.908411,-0.898287,0.626713
A2ML1,-0.495620,-0.626909,-0.384588,-0.838927,-0.939908,-0.970160,-0.435073,0.433457,-0.606711,-0.788458,...,-0.223009,0.201126,0.069845,0.756926,0.574924,-0.010932,0.019318,0.483945,0.130440,-0.344186
A3GALT2,0.701807,0.792815,-0.752632,-0.540598,-0.863703,-0.530520,0.661392,0.327850,-0.944470,0.297522,...,-0.338627,0.843345,0.085475,0.873715,0.499586,0.206651,0.398571,0.742262,-0.389146,-0.550684


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [64]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [65]:
filename = path+'tcga_esca_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [66]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [67]:
tertiary_matrix.head()

,TCGA-2H-A9GF-01,TCGA-2H-A9GG-01,TCGA-2H-A9GH-01,TCGA-2H-A9GI-01,TCGA-2H-A9GJ-01,TCGA-2H-A9GK-01,TCGA-2H-A9GL-01,TCGA-2H-A9GM-01,TCGA-2H-A9GN-01,TCGA-2H-A9GO-01,...,TCGA-VR-AA7D-01,TCGA-VR-AA7I-01,TCGA-X8-AAAR-01,TCGA-XP-A8T6-01,TCGA-XP-A8T8-01,TCGA-Z6-A8JD-01,TCGA-Z6-A8JE-01,TCGA-Z6-A9VB-01,TCGA-Z6-AAPN-01,TCGA-ZR-A9CJ-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [68]:
filename = path+'tcga_esca_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [69]:
name = 'tcga_esca_gene_up_set'

In [70]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [71]:
name = 'tcga_esca_gene_down_set'

In [72]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [73]:
name = 'tcga_esca_attribute_up_set'

In [74]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [75]:
name = 'tcga_esca_attribute_down_set'

In [76]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity matrix

In [77]:
attribute_normalized_matrix = normalized_matrix.copy()

In [78]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [79]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [80]:
attribute_similarity_matix.head()

,"(Donor:TCGA-2H-A9GF, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG3)","(Donor:TCGA-2H-A9GG, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-2H-A9GH, Age:44.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-2H-A9GI, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-2H-A9GJ, Age:57.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-2H-A9GK, Age:43.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG3)","(Donor:TCGA-2H-A9GL, Age:74.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG3)","(Donor:TCGA-2H-A9GM, Age:53.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-2H-A9GN, Age:70.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG3)","(Donor:TCGA-2H-A9GO, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IVA, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG3)",...,"(Donor:TCGA-VR-AA7D, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIC, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-VR-AA7I, Age:70.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG1)","(Donor:TCGA-X8-AAAR, Age:69.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG1)","(Donor:TCGA-XP-A8T6, Age:54.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-XP-A8T8, Age:49.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG1)","(Donor:TCGA-Z6-A8JD, Age:53.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIB, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-Z6-A8JE, Age:57.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG3)","(Donor:TCGA-Z6-A9VB, Age:53.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIA, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG2)","(Donor:TCGA-Z6-AAPN, Age:57.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIA, Histology:Esophagus Squamous Cell Carcinoma, Tissue:Esophagus, Tumor:nan, GradeG1)","(Donor:TCGA-ZR-A9CJ, Age:65.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IIIC, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG3)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-2H-A9GF, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Esophagus Adenocarcinoma, NOS, Tissue:Esophagus, Tumor:nan, GradeG3)",1.000000,0.078104,0.138992,0.098240,0.108052,0.082829,0.043485,0.084855,0.086074,0.047047,...,0.005730,0.006346,0.052231,0.067503,0.015077,0.077031,0.017662,0.030911,-0.008440,0.054004
"(Donor:TCGA-2H-A9GG, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Es

# Save Attribute Similarity Matrix

In [81]:
filename = path+'tcga_esca_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [82]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

# Create Gene Similarity Matrix

In [83]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [84]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.059782,-0.191926,-0.041075,-0.151492,-0.003502,-0.100936,0.103020,0.076307,0.068030,...,0.033705,-0.059099,-0.014990,-0.181071,-0.021114,0.025662,-0.124658,-0.020116,-0.059102,0.024550
A1CF,-0.059782,1.000000,0.172306,-0.463047,-0.059767,-0.590872,0.264826,-0.108649,-0.156116,0.207852,...,-0.061578,0.153193,-0.084834,0.161005,-0.125882,-0.318544,-0.120004,-0.192825,0.298942,-0.063926
A2M,-0.191926,0.172306,1.000000,-0.295766,0.059266,-0.086083,0.178318,-0.266816,-0.305798,0.029799,...,-0.439850,-0.312761,0.052320,0.152723,-0.207526,-0.261584,0.264716,0.211991,0.295584,0.123983
A2ML1,-0.041075,-0.463047,-0.295766,1.000000,0.037736,0.438498,-0.181840,-0.084752,0.341115,-0.136263,...,0.018037,-0.119850,-0.025337,-0.253548,0.138611,0.280323,0.060926,0.064420,-0.137112,-0.144456
A3GALT2,-0.151492,-0.059767,0.059266,0.037736,1.000000,-0.055268,-0.040458,-0.116485,0.107620,-0.008536,...,0.008556,0.080214,0.028559,0.079102,0.064719,0.087224,0.154864,-0.022339,0.067623,0.088420


# Save Gene Similarity Matrix

In [85]:
filename = path+'tcga_esca_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [86]:
name = 'tcga_esca_gene_attribute_edge_list'

In [87]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  198 Out of 198   

 The number of statisticaly relevent gene-attribute associations is: 186087
